In [1]:
import time
import numpy as np

In [61]:
# Load Lambda dataset
import json

json_file_name = 'section7_Lambda.json'
with open(json_file_name, 'r') as json_file:
    Lambda = [tuple(sorted(x)) for x in json.load(json_file)]
print(f"Loaded JSON Lambda. It contains {len(Lambda)} items.")

Loaded JSON Lambda. It contains 2289 items.


### relcoord

$\mathrm{relcoord}: \{0, ..., 59 \} \to \{0,1,2,3,M,d-6,...,d \}^2$

In [3]:
def relcoord(index):
    """
    index: index in super contracted coordinate system
    """
    M = 500
    d = 1000
    indexes = { 
        'x': set(range(16)),
        'y': set(range(16, 32)),
        'z': set(range(32, 48)),
        'b': set(range(48, 52)),
        'c': set(range(52, 56)),
        'd': set(range(56, 60)),
    }
    if index in indexes['x']:
        col = index // 4
        row = index % 4
        return [(col, row)]
    elif index in indexes['y']:
        col = (index - 16) // 4
        row = (-3 + index % 4) - col + d
        return [(col, row)]
    elif index in indexes['z']:
        row = index % 4
        col = d + (-3 + (index - 32) // 4) - row
        return [(col, row)]
    elif index in indexes['b']:
        b_index = index - 48
        return [(M, b_index)] + [(d-6 + i , b_index) for i in range(3 - b_index)]
    elif index in indexes['c']:
        c_index = index - 52
        return [(c_index, M)] + [(c_index, d-6 + i) for i in range(3 - c_index)]
    elif index in indexes['d']:
        d_index = index - 56
        return [(M, M)] + [(M, d-6 + i) for i in range(3 - d_index)] + [(d-6 + i, M) for i in range(3 - d_index)]
    else:
        raise Value

def stringify_relcoord(x):
    if x == 500:
        return 'M'
    elif x == 1000:
        return 'd'
    elif x > 500:
        return f"d{x - 1000}"
    else:
        return x

def print_relcoord(coord):
    print([(stringify_relcoord(x), stringify_relcoord(y)) for (x,y) in coord])

In [60]:
# Test relcoord
print_relcoord(relcoord(56))

[('M', 'M'), ('M', 'd-6'), ('M', 'd-5'), ('M', 'd-4'), ('d-6', 'M'), ('d-5', 'M'), ('d-4', 'M')]


### relsets
The following function `relsets` computes for a super contracted configuration $v$ the set $\{ \mathrm{relcoord}(w) : \exists w \ \in \mathbb Z^{V_d} \text{ such that } \mathrm{contr}'(sign(w)) = v \}$

In [5]:
def relsets(config):
    """
    Find all configurations in Z^Vd in relative coordinate system 
    that map to super_contracted_config under (contract'◦sign)
    """
    assert all([True if 0 <= x and x < 60 else False for x in config]), "Passed argument is not in valid super contracted form"

    rel_config = [relcoord(x) for x in config]
    accu = []
    res = []
    def dfs(index):
        if index >= len(rel_config):
            res.append(accu.copy())
            return
        for x in rel_config[index]:
            accu.append(x)
            dfs(index + 1)
            accu.pop()
    
    dfs(0)
    return res

In [58]:
# Test relsets
lambd = Lambda[100]

print(f"Super contracted configuration {Lambda[100]}")
print("Here is a list of configurations in Z^Vd that map to this super contracted configuration:")
print()
for r in relsets(lambd):
    print([(stringify_relcoord(x), stringify_relcoord(y)) for x,y in r])

print()
print("Configurations are given in relative coordinates.")

Super contracted configuration (12, 19, 20, 45, 56)
Here is a list of configurations in Z^Vd that map to this super contracted configuration:

[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('M', 'M')]
[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('M', 'd-6')]
[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('M', 'd-5')]
[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('M', 'd-4')]
[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('d-6', 'M')]
[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('d-5', 'M')]
[(3, 0), (0, 'd'), (1, 'd-4'), ('d-1', 1), ('d-4', 'M')]

Configurations are given in relative coordinates.


## Apply Invertibility Criterion using relative coordinates

We use the divide and conquer approach.

### Divide

In [145]:
SENTINEL_M = 500
SENTINEL_D = 1000
REL = [0,1,2,3,SENTINEL_M,SENTINEL_D-6,SENTINEL_D-5,SENTINEL_D-4,SENTINEL_D-3,SENTINEL_D-2,SENTINEL_D-1,SENTINEL_D]

def col(p):
    return p[0]

def row(p):
    return p[1]

def divide(relset):
    """
    Computes the lambda for the division step. 
    Returns lambda if a division was found, otherwise None is returned.

    Note that if None is returned, it does not necessarily mean that there is no division. 
    It just means that this (simple) algorithm could not find a division,
    but there may exist one, and we haven't found it.

    relset: support of a configuration in relative coordinates
    """
    M = 500
    d = 1000
    R = [0, 1, 2, 3, M, d-6, d-5, d-4, d-3, d-2, d-1, d] # relative coordinate system

    assert len(relset) == 6 #size of negative support + size of negative support = 6
    assert (0,0) in relset

    lambd = []
    col_start = 0
    for col_end in range(len(R)):
        num_cols = col_end - col_start + 1
        points_in_col = [p for p in relset if col(p) in range(R[col_start], R[col_end]+1)]
        num_points = len(points_in_col)

        assert not num_points or num_cols <= num_points

        if not num_points or num_cols == num_points or R[col_end] == M:
            lambd.append(num_points)
            col_start = col_end + 1
            
    if sum(lambd) != 6:
        return None
        
    return lambd

In [146]:
# Test divide
l = Lambda[400]
for relset in relsets(l):
    config = [(0,0)] + relset
    config_str = str(sorted([(stringify_relcoord(x), stringify_relcoord(y)) for x,y in config], key=lambda x: str(x[0])))
    lambd = divide(config)
    print(f"Computed lambda={lambd} for config {config_str}.")

Computed lambda=[3, 0, 2, 0, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 1), ('M', 'M'), ('d', 0)].
Computed lambda=[3, 0, 2, 0, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 1), ('M', 'd-6'), ('d', 0)].
Computed lambda=[3, 0, 2, 0, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 1), ('M', 'd-5'), ('d', 0)].
Computed lambda=[3, 0, 1, 1, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 1), ('d', 0), ('d-6', 'M')].
Computed lambda=[3, 0, 1, 0, 1, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 1), ('d', 0), ('d-5', 'M')].
Computed lambda=[3, 0, 1, 1, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 'M'), ('d', 0), ('d-6', 1)].
Computed lambda=[3, 0, 1, 1, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 'd-6'), ('d', 0), ('d-6', 1)].
Computed lambda=[3, 0, 1, 1, 0, 0, 0, 0, 0, 1] for config [(0, 0), (0, 3), (1, 'd-1'), ('M', 'd-5'), ('d', 0), ('d-6', 1)].
Computed lambda=[3, 0,

In [147]:
for l in Lambda:
    for relset in relsets(l):
        config = [(0,0)] + relset
        config_str = str(sorted([(stringify_relcoord(x), stringify_relcoord(y)) for x,y in config], key=lambda x: str(x[0])))
        lambd = divide(config)
        if not lambd:
            print(f"Divide step failed for config={config_str}")
print("Divide step finished")

Divide step finished


### Conquer

In [233]:
SENTINEL_M = 500
SENTINEL_D = 1000

def col(p):
    return p[0]

def row(p):
    return p[1]

def succ(x):
    if x == 3:
        return SENTINEL_M
    if x == SENTINEL_M:
        return SENTINEL_D-6
    return x+1

def midpoint(a,b) -> list[int]:
    low, high = min(a,b), max(a,b)

    # low and high take concrete values
    # hence we can compute the midpoint exactly
    if low > SENTINEL_M or high < SENTINEL_M: 
        return [(low + high - 1) / 2]
        
    if high == SENTINEL_M:
        if low in [0,1]:
            return [2,3,SENTINEL_M]
        if low in [2,3]:
            return [3,SENTINEL_M]
    if low == SENTINEL_M:
        if high in [SENTINEL_D-6, SENTINEL_D-5]:
            return [SENTINEL_M]
        if high in [SENTINEL_D-4, SENTINEL_D-3]:
            return [SENTINEL_M,SENTINEL_D-6]
        if high in [SENTINEL_D-2, SENTINEL_D-1]:
            return [SENTINEL_M,SENTINEL_D-6,SENTINEL_D-5]       
        if high == SENTINEL_D:
            return [SENTINEL_M,SENTINEL_D-6,SENTINEL_D-5,SENTINEL_D-4]
    
    return [SENTINEL_M]
        
def conquer(relset):
    """
    Returns True if invertibility criterion can be applied, otherwise False
    """
    relset = sorted(relset, key=lambda x: col(x))
    length = len(relset)
    if length <= 2:
        return True
    elif length == 3:
        x, y, z = relset
        same_column = col(x) == col(y) and col(y) == col(z)
        
        if same_column: # Proposition 5.7c
            return True

        if col(x) == col(y) and succ(col(x)) == col(z): # Proposition 5.7d
            if row(z) not in midpoint(row(x),row(y)): # see Proposition 5.7d
                return True
            
        return False
    else:
        return False

### Apply

In [237]:
# helper function
def make_subrelset(relset, lambd):
    nonzero = [l for l in lambd if l > 0]
    begin = 0
    for n in nonzero:
        yield relset[begin:begin+n]
        begin += n

def apply_invertibility_criterion(relset):
    assert len(relset) == 6 #size of negative support + size of negative support = 6
    assert (0,0) in relset
    
    relset = sorted(relset, key=lambda x: col(x))
    lambd = divide(relset)
    conquer_success = all([conquer(subrelset) for subrelset in make_subrelset(relset, lambd)])

    return conquer_success

In [236]:
left_to_check = []
for l in Lambda:
    res_inv_crit = [apply_invertibility_criterion([(0,0)] + r) for r in relsets(l)]
    if not all(res_inv_crit):
        left_to_check.append(l)
len(left_to_check)

1107